In [52]:
#Takes in xy coordinates of jetbot and goal, and outputs command to jetbot.

#from jetbot import Robot
#robot = Robot()
import numpy as np
import math
import time

In [53]:
def orientate (jetPos, grabberPos, goalPos): #takes in xy coords of jetbot center of mass,
#jetbot grabber (or other jetbot feature), and target, and returns angle to target in radians
    #Set jetbot position as origin
    absGrabberPos = tuple(map(lambda i, j: i - j, grabberPos, jetPos))
    absGoalPos = tuple(map(lambda i, j: i - j, goalPos, jetPos))
    
    #Find angle between grabber (the direction the jet's facing) and the goal
    #Code from https://www.kite.com/python/answers/how-to-get-the-angle-between-two-vectors-in-python#
    vector_1 = absGrabberPos
    vector_2 = absGoalPos
    
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1) #Make unit vectors, length = 1
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    
    return angle

In [54]:
# Test orientate
orientate((0,0), (0,1), (1,0))

1.5707963267948966

In [55]:
def correctionCommand(angle):
    if angle < math.pi and angle > math.pi /4:
        return "right"
    if angle >= math.pi and angle <math.pi *7/4:
        return "left"
    else: #if angle is between 7pi/4 and pi/4, or within 45 degrees of target
        return "forward"

In [56]:
correctionCommand(math.pi)

'left'

In [57]:
jetPos = (10,10)
grabberPos = (9,9)
goalPos = (20,0)
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '192.168.1.30' #Set this to be your IP address, found in ipconfig or socket.gethostbyname(socket.gethostname())
s.bind((host, 1234))
s.listen(5)

while True:
    #Collect coords somehow
    distance = math.dist(jetPos, goalPos)
    print("Distance:", end = " ")
    print(distance)
    if distance < 1: #Repeat until goal is reached
        break #end loop
    
    angle = orientate(jetPos, grabberPos, goalPos)
    print("Angle:", end = " ")
    print(angle)
    
    command = correctionCommand(angle)
    print(command)
    
    clientsocket, address = s.accept() #Waits until jetbot makes connection
    print(f"Connection from {address} has been established.")
    clientsocket.send(bytes(command,"utf-8")) #Sends command to the jetbot
    time.sleep(1)
    
    

OSError: [Errno 48] Address already in use